In [1]:
use Dan;
use Dan::Pandas;

In [40]:
# Read .csv file (Text::CSV)

use Text::CSV;
my @lines  = csv( in => 'DemographicData.csv' );

my \stats = DataFrame.new( columns => @lines[0], data => @lines[1..*-1] );
~stats[0..3]^;      # head

  Country Name Country Code Birth rate Internet users         Income Group
0        Aruba          ABW     10.244           78.9          High income
1  Afghanistan          AFG     35.253            5.9           Low income
2       Angola          AGO     45.985           19.1  Upper middle income
3      Albania          ALB     12.877           57.2  Upper middle income

In [3]:
stats.^name;

Dan::Pandas::DataFrame

In [4]:
# '~' makes a Str, '^' makes a DataFrame from rows

~stats[*-5..*-1]^;    # tail

         Country Name Country Code Birth rate Internet users         Income Group
190       Yemen, Rep.          YEM     32.947             20  Lower middle income
191      South Africa          ZAF      20.85           46.5  Upper middle income
192  Congo, Dem. Rep.          COD     42.394            2.2           Low income
193            Zambia          ZMB     40.471           15.4  Lower middle income
194          Zimbabwe          ZWE     35.715           18.5           Low income

In [5]:
stats.shape;

[195 5]

In [6]:
stats.elems;

195

In [7]:
stats.columns.elems;  

5

In [19]:
#Clean up #1 - tidy up col labels

stats.pd: ".columns = ['CountryName', 'CountryCode', 'BirthRate', 'InternetUsers', 'IncomeGroup']";
say stats.columns;     #Hash (unordered)
say stats.cx;          #Seq    (ordered)

{BirthRate => 2, CountryCode => 1, CountryName => 0, IncomeGroup => 4, InternetUsers => 3}
[CountryName CountryCode BirthRate InternetUsers IncomeGroup]


In [ ]:
## FIXME work this out

##Clean up #2 - coerce cols to numbers      [see DD-raku-Dan-typed.ipynb for why & when]

#stats.data[*;2,3].map({$_.=Rat});
#stats.dtypes;

In [42]:
# This cell exports the Dan::Pandas::DataFrame for use in DD-raku-Dan-to-seaborn.ipynb

stats.pd: ".to_json('DD-done.json')";

In [36]:
stats.dtypes;      #FIXME - SO to cell

In [20]:
stats.describe;    #FIXME - SO to cell

In [38]:
~stats.series: <BirthRate>;

0      10.244
1      35.253
2      45.985
3      12.877
4      11.044
        ...  
190    32.947
191     20.85
192    42.394
193    40.471
194    35.715
Name: BirthRate, Length: 195, dtype: object

In [21]:
~stats[0..5]<CountryName BirthRate>;

            CountryName BirthRate
0                 Aruba    10.244
1           Afghanistan    35.253
2                Angola    45.985
3               Albania    12.877
4  United Arab Emirates    11.044
5             Argentina    17.716

In [22]:
~stats[185..*-1]^;       # Range operator (start .. end)

               CountryName CountryCode BirthRate InternetUsers          IncomeGroup
185  Virgin Islands (U.S.)         VIR      10.7          45.3          High income
186                Vietnam         VNM    15.537          43.9  Lower middle income
187                Vanuatu         VUT    26.739          11.3  Lower middle income
188     West Bank and Gaza         PSE    30.394          46.6  Lower middle income
189                  Samoa         WSM    26.172          15.3  Lower middle income
190            Yemen, Rep.         YEM    32.947            20  Lower middle income
191           South Africa         ZAF     20.85          46.5  Upper middle income
192       Congo, Dem. Rep.         COD    42.394           2.2           Low income
193                 Zambia         ZMB    40.471          15.4  Lower middle income
194               Zimbabwe         ZWE    35.715          18.5           Low income

In [23]:
#Reverse order
~stats[10,*-1...5]^;     # Sequence operator (start,operation ... end)

            CountryName CountryCode BirthRate InternetUsers          IncomeGroup
10           Azerbaijan         AZE      18.3          58.7  Upper middle income
9               Austria         AUT       9.4       80.6188          High income
8             Australia         AUS      13.2            83          High income
7   Antigua and Barbuda         ATG    16.447          63.4          High income
6               Armenia         ARM    13.308          41.9  Lower middle income
5             Argentina         ARG    17.716          59.9          High income

In [24]:
#Every 20th item
~stats[0,20...stats.elems]^;

    CountryName CountryCode BirthRate InternetUsers          IncomeGroup
0         Aruba         ABW    10.244          78.9          High income
20      Belarus         BLR      12.5         54.17  Upper middle income
40   Costa Rica         CRI    15.022         45.96  Upper middle income
60        Gabon         GAB    30.555           9.2  Upper middle income
80        India         IND    20.291          15.1  Lower middle income
100       Libya         LBY    21.425          16.5  Upper middle income
120  Mozambique         MOZ    39.705           5.4           Low income
140      Poland         POL       9.6       62.8492          High income
160    Suriname         SUR    18.455          37.4  Upper middle income
180     Uruguay         URY    14.374         57.69          High income

In [25]:
#Calculate and Append a new column

my $calcs = Series.new( stats[*]<BirthRate> >>*<< stats[*]<InternetUsers> );   #multiply every row

stats.splice( :ax, *, *, [MyCalc => $calcs] );
~stats[0..5]^;

            CountryName CountryCode BirthRate InternetUsers          IncomeGroup     MyCalc
0                 Aruba         ABW    10.244          78.9          High income   808.2516
1           Afghanistan         AFG    35.253           5.9           Low income   207.9927
2                Angola         AGO    45.985          19.1  Upper middle income   878.3135
3               Albania         ALB    12.877          57.2  Upper middle income   736.5644
4  United Arab Emirates         ARE    11.044            88          High income   971.8720
5             Argentina         ARG    17.716          59.9          High income  1061.1884

In [26]:
#Drop column

stats.splice( :ax, *-1 );
~stats[0..5]^;

            CountryName CountryCode BirthRate InternetUsers          IncomeGroup
0                 Aruba         ABW    10.244          78.9          High income
1           Afghanistan         AFG    35.253           5.9           Low income
2                Angola         AGO    45.985          19.1  Upper middle income
3               Albania         ALB    12.877          57.2  Upper middle income
4  United Arab Emirates         ARE    11.044            88          High income
5             Argentina         ARG    17.716          59.9          High income

In [27]:
# Grep (check whether column 3 value < 2)

~stats.grep( { .[3] < 2 } );

      CountryName CountryCode BirthRate InternetUsers          IncomeGroup
11        Burundi         BDI    44.151           1.3           Low income
52        Eritrea         ERI      34.8           0.9           Low income
55       Ethiopia         ETH    32.925           1.9           Low income
64         Guinea         GIN    37.337           1.6           Low income
117       Myanmar         MMR    18.119           1.6  Lower middle income
127         Niger         NER    49.661           1.7           Low income
154  Sierra Leone         SLE    36.729           1.7           Low income
156       Somalia         SOM    43.891           1.5           Low income
172   Timor-Leste         TLS    35.755           1.1  Lower middle income

In [28]:
# Grep (simple col label check)

~stats.grep( { stats[$++]<InternetUsers> < 2 } );

      CountryName CountryCode BirthRate InternetUsers          IncomeGroup
11        Burundi         BDI    44.151           1.3           Low income
52        Eritrea         ERI      34.8           0.9           Low income
55       Ethiopia         ETH    32.925           1.9           Low income
64         Guinea         GIN    37.337           1.6           Low income
117       Myanmar         MMR    18.119           1.6  Lower middle income
127         Niger         NER    49.661           1.7           Low income
154  Sierra Leone         SLE    36.729           1.7           Low income
156       Somalia         SOM    43.891           1.5           Low income
172   Timor-Leste         TLS    35.755           1.1  Lower middle income

In [29]:
# Grep (combine two checks with col labels)

~stats.grep( { my $i=stats[$++]; $i.<InternetUsers> < 2 && $i.<BirthRate> > 40 } );

    CountryName CountryCode BirthRate InternetUsers IncomeGroup
11      Burundi         BDI    44.151           1.3  Low income
127       Niger         NER    49.661           1.7  Low income
156     Somalia         SOM    43.891           1.5  Low income

In [30]:
# Grep (define filters and apply later)

sub filter1 { $^i.<InternetUsers> < 2  };
sub filter2 { $^i.<BirthRate>     > 40 };

~stats.grep( { my $i=stats[$++]; $i.&filter1 && $i.&filter2 } );

    CountryName CountryCode BirthRate InternetUsers IncomeGroup
11      Burundi         BDI    44.151           1.3  Low income
127       Niger         NER    49.661           1.7  Low income
156     Somalia         SOM    43.891           1.5  Low income

In [31]:
# Grep with regex

stats.grep( { stats[$++]<IncomeGroup> ~~ /High/ } ).elems;

67

In [32]:
# Unique

stats[*]<IncomeGroup>.unique.join(', ');

High income, Low income, Upper middle income, Lower middle income

In [33]:
# Accessors

stats[4][3];

88

In [34]:
stats[4]<InternetUsers>;

88

In [35]:
stats<4>[3];   #The default index is letter 4 => 4 (Int) so you may use either [] or <> accessor

88